In [42]:
import Bio
from Bio import Entrez
import pandas as pd
import os
os.chdir('..')

In [81]:
# Entrez.email = EMAIL_HERE
# Entrez.api_key = KEY_HERE
Bio.Entrez.max_tries = 2
Bio.Entrez.sleep_between_tries = 5

start_date = '2022/05/12'
end_date = '2022/12/27'

handle = Entrez.esearch("taxonomy", start_date + ":" + end_date + "[edat] AND Bacteria[SubTree] AND species[Rank] NOT unclassified[prop] NOT uncultured[prop] AND (\"above species level\"[prop] OR specified[prop])", retmax = 1000)
record = Entrez.read(handle)
handle.close()
taxIDs = record["IdList"]

In [82]:
nodes = pd.read_csv('ncbi_taxdump/nodes.dmp', sep='\t', header=None).iloc[:, [0,2,4]]
nodes.columns = ['TaxID', 'ParentID', 'Rank']

/var/folders/sd/kt91y68x51v580jtrx05xmjw0000gn/T/ipykernel_33416/963291618.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  nodes = pd.read_csv('ncbi_taxdump/nodes.dmp', sep='\t', header=None).iloc[:, [0,2,4]]


In [83]:
i = 0
while i < len(taxIDs):
    parent_id = nodes[nodes['TaxID'] == int(taxIDs[i])]['ParentID'].tolist()[0]
    if len(nodes[nodes['ParentID'] == int(parent_id)]['ParentID'].tolist()) == 1:
        if len(nodes[nodes['ParentID'] == int(parent_id)]['ParentID'].tolist()) == 0:
            raise ValueError("Taxa missing")
        taxIDs.append(parent_id)
    i += 1

In [101]:
unknown_taxa = [int(taxID) for taxID in taxIDs]
pd.DataFrame.from_dict({'IDs':unknown_taxa}).to_csv('ncbi_taxdump/unknown_taxa.tsv', sep='\t', header=None, index = None)